## Demo notebook to calssify pneumonia

The data is taken from <br>
https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data



imports ...

In [1]:
import os 
import torch
from torch import nn, optim
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as v2
import math
import time 
import matplotlib.pyplot as plt

Parametes 

In [2]:
dataset_folder  = "Dataset"
train_folder = "train"
valid_folder = "val"
test_folder = "test"
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print (f"Settings ... \n dataset path \t\t {dataset_folder} \n batch_size \t\t {batch_size}" +
       f" \n device \t\t {device}")

Using device: cuda
Settings ... 
 dataset path 		 Dataset 
 batch_size 		 32 
 device 		 cuda


Check the classes in the folder

In [3]:
class_labels = [name for name in os.listdir(f"{dataset_folder}/{train_folder}") if not name.startswith(".DS_Store")]
print(class_labels)
num_classes = len(class_labels)
print(num_classes)

['NORMAL', 'PNEUMONIA']
2


Creating dataset from the image folder 

In [4]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transformations = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomResizedCrop(size=(256, 256), antialias=True),
    v2.RandomRotation(degrees = (0, 170)),
    v2.ToDtype(dtype = torch.float32, scale = True),
    v2.Resize(size = (256, 256), antialias = True),
    v2.Normalize(mean, std)
])

train_dataset = ImageFolder(f"{dataset_folder}/{train_folder}", transform = transformations)
valid_dataset = ImageFolder(f"{dataset_folder}/{valid_folder}", transform = transformations)
test_dataset = ImageFolder(f"{dataset_folder}/{test_folder}", transform = transformations)

## probe stats from the Image folder 

In [5]:
print (f"classes {train_dataset.classes}")
d = train_dataset.class_to_idx

num_classes =2

classes ['NORMAL', 'PNEUMONIA']


## Data loader time 

In [6]:


train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset,shuffle=True,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,shuffle=True, batch_size=batch_size)

print(f"Lenght of training dataset {len(train_dataloader.dataset)}")
print(f"Lenght of validation dataset {len(valid_dataloader.dataset)}")
print(f"Lenght of test dataset {len(test_dataloader.dataset)}")

Lenght of training dataset 5216
Lenght of validation dataset 16
Lenght of test dataset 624


try ResNet 

In [7]:
res_net = torchvision.models.resnet50(weights='IMAGENET1K_V1')

for param in res_net.parameters():
    param.requires_grad = False

res_net.fc  = torch.nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(
        in_features=128,
        out_features=num_classes
    ),
)



ANy other model(s) to evalaute can go here ... 

In [8]:
model_to_evaluate = res_net

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(res_net.fc.parameters())

training method 

In [10]:
def train_model(model,model_name,criterion,optimier, data_loader,device, num_epochs=0):
    model.to(device)
    for epoch in range(num_epochs):
        #loss_batches = 0
        loss_epoch=0;
        corrects_batches = 0
        count = 0
        start = time.time()
        correct = 0
        total = 0
        data_loading_begin = time.time()
        data_loading_time = 0
        data_processing_time = 0 
        for x,y in data_loader:
            
            x,y = x.to(device), y.to(device)
            data_loading_time += (time.time()-data_loading_begin)
            processing_time_begin = time.time()
            outputs = model(x)
            loss = criterion(outputs,y)
            optimier.zero_grad()
            loss.backward()
            optimier.step()
            data_processing_time += (time.time()-processing_time_begin)
            _,preds = torch.max(outputs,1)

            correct += (preds == y).sum().item()
            total += preds.size(0)
            loss_epoch += loss.item()
            count += 1
            data_loading_begin = time.time()
        #epoch_loss = loss_batches / len(data_loader)
        epoch_acc = correct / total
        print(f"\n epoch {epoch} Loss : {(loss_epoch/count):.4f} Accuracy {epoch_acc:.2f}, time : {time.time()-start} secs")
        print(f"data loading time {data_loading_time} secs, data processing time {data_processing_time} secs")
        if (epoch % 3 == 0 ):
          torch.save(model.state_dict(),f"{model_name}_{epoch:02d}_{epoch_acc:.2f}.h5")
    return model



In [11]:
train_model(model_to_evaluate,"res_net",criterion=criterion, optimier=optimizer,
            data_loader=train_dataloader, device=device, num_epochs=30)

torch.save(model_to_evaluate.state_dict(),'resNet.h5')


 epoch 0 Loss : 0.3612 Accuracy 0.84, time : 516.2087528705597 secs
data loading time 467.64172530174255 secs, data processing time 39.198058128356934 secs

 epoch 1 Loss : 0.3355 Accuracy 0.85, time : 226.7957444190979 secs
data loading time 215.22204685211182 secs, data processing time 3.7457752227783203 secs

 epoch 2 Loss : 0.3128 Accuracy 0.86, time : 223.42679500579834 secs
data loading time 211.8066189289093 secs, data processing time 3.8180460929870605 secs

 epoch 3 Loss : 0.2983 Accuracy 0.87, time : 251.41879224777222 secs
data loading time 239.6731083393097 secs, data processing time 3.717101812362671 secs

 epoch 4 Loss : 0.3029 Accuracy 0.87, time : 357.66844034194946 secs
data loading time 345.3748390674591 secs, data processing time 3.828052043914795 secs

 epoch 5 Loss : 0.2890 Accuracy 0.87, time : 283.5943007469177 secs
data loading time 272.01806807518005 secs, data processing time 3.81483793258667 secs

 epoch 6 Loss : 0.2902 Accuracy 0.87, time : 240.404470682144

Load model 

In [7]:
res_net = torchvision.models.resnet50()

res_net.fc  = torch.nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(
        in_features=128,
        out_features=num_classes
    ),
)

res_net.load_state_dict(torch.load('resNet.h5'))

NameError: name 'num_classes' is not defined

In [3]:
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x,y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            correct += (predictions == y).sum().item()
            total += predictions.size(0)
    model.train()
    return correct/total

In [4]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(20,20))
  for n in range(len(image_batch)):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(label_batch[n])
      plt.axis('off')

In [5]:
image_batch, label_batch = next(iter(train_dataloader))
show_batch(image_batch, label_batch)

NameError: name 'train_dataloader' is not defined

In [6]:
print(f"Training Accuracy is {calculate_accuracy(train_dataloader, res_net)*100}")

print(f"Testing Accuracy is {calculate_accuracy(test_dataloader, res_net)*100}")

NameError: name 'train_dataloader' is not defined